# MiniDataAPI Spec

The `MiniDataAPI` is a persistence API specification that designed to be small and relatively easy to implement across a wide range of datastores. While early implementations have been SQL-based, the specification can be quickly implemented in key/value stores, document databases, and more.

In [229]:
#| hide
from fasthtml.common import *

## Connect/construct the database

We connect or construct the database by passing in a string connecting to the database endpoint or a filepath representing the database's location. While this example is for a SQLite running in memory, other databases such as PostgreSQL, Redis, MongoDB, would instead use a URI pointing at the database's filepath or endpoint.

In [230]:
db = database('sqlite:///:memory:')

## Creating the tables

We use a `create()` method attached to `Database` object (`db` in our example) to create the tables. Depending on the database type, this method can include transforms - the ability to modify the tables.

In [231]:
class User: name:str; email: str; year_started:int
users = db.create(User, pk='name')
users

<Table user (name, email, year_started)>

From the table objects we can extract a Dataclass version of our tables. Usually this is given an singular uppercase version of our table name, which in this case is `User`.

In [232]:
UserDC = users.dataclass()

If no `pk` is provided, id is assumed to be the primary key.

In [233]:
class Todo: id: int; title: str; detail: str; status: str; name: str
todos = db.create(Todo) 
todos

<Table todo (id, title, detail, status, name)>

Let's extract the dataclass for `Todo`.

In [234]:
TodoDC = todos.dataclass()

In [235]:
#| hide
# Remove records
[users.delete(o['name']) for o in users.rows]
[todos.delete(o['id']) for o in todos.rows]

[<Table todo (id, title, detail, status, name)>,
 <Table todo (id, title, detail, status, name)>,
 <Table todo (id, title, detail, status, name)>]

## .insert()

Add a new record to the database. We want to support as many types as possible, for now we have tests for Python classes, dataclasses, and dicts. Returns an instance of the new record.

Here's how to add a record using a Python class.

In [236]:
users.insert(User(name='Braden', email='b@example.com', year_started=2018))



User(name='Braden', email='b@example.com', year_started=2018)

Let's add another user, this time via a dataclass.

In [237]:
users.insert(UserDC(name='Alma', email='a@example.com', year_started=2019))

User(name='Alma', email='a@example.com', year_started=2019)

And now Charlie gets added via a Python dict.

In [238]:
users.insert({'name': 'Charlie', 'email': 'c@example.com', 'year_started': 2018})

User(name='Charlie', email='c@example.com', year_started=2018)

And now all three methods for TODOs

In [239]:

todos.insert(Todo(title='Write MiniDataAPI spec', status='open', name='Braden'))
todos.insert(TodoDC(title='Implement SSE in FastHTML', status='open', name='Alma'))
todo = todos.insert(dict(title='Launch FastHTML', status='closed', name='Charlie'))
todo

Todo(id=3, title='Launch FastHTML', detail=None, status='closed', name='Charlie')

## Square bracket search []

Get a single record by entering a primary key into a table object within square brackets. Let's see if we can find Alma.

In [240]:
users['Alma']

User(name='Alma', email='a@example.com', year_started=2019)

If no record is found, a `NotFoundError` error is raised. Here we look for David, who hasn't yet been added to our users table.

In [241]:
try: users['David']
except NotFoundError: print(f'User not found')

User not found


Here's a demonstration of a ticket search, demonstrating how this works with non-string primary keys. 

In [242]:
todos[1]

Todo(id=1, title='Write MiniDataAPI spec', detail=None, status='open', name='Braden')

## Parentheses search ()

Get zero to many records by entering values with parentheses searches. If nothing is in the parentheses, then everything is returned. 

In [243]:
users()

[User(name='Braden', email='b@example.com', year_started=2018),
 User(name='Alma', email='a@example.com', year_started=2019),
 User(name='Charlie', email='c@example.com', year_started=2018)]

We can order the results.

In [244]:
users()

[User(name='Braden', email='b@example.com', year_started=2018),
 User(name='Alma', email='a@example.com', year_started=2019),
 User(name='Charlie', email='c@example.com', year_started=2018)]

We can filter on the results:

In [245]:
users()

[User(name='Braden', email='b@example.com', year_started=2018),
 User(name='Alma', email='a@example.com', year_started=2019),
 User(name='Charlie', email='c@example.com', year_started=2018)]

## .update()

Update an existing record of the database. Must accept Python dict, dataclasses, and standard classes. Uses the primary key for identifying the record to be changed. Returns an instance of the updated record. 

Here's with a normal Python class:

In [246]:
users.update(User(name='Alma', year_started=2099, email='a@example.com'))


User(name='Alma', email='a@example.com', year_started=2099)

Now with a dataclass, sending Alma further in time:

In [247]:
users.update(UserDC(name='Alma', year_started=2199, email="a@example.com"))

User(name='Alma', email='a@example.com', year_started=2199)

If the primary key doesn't match a record, raise a `NoteFoundError`. 

John hasn't started with us yet so doesn't get the chance yet to travel in time.

In [248]:
try: users.update(User(name='John', year_started=2024, email='j@example.com'))
except NotFoundError: print('User not found')

User not found


## .delete()

Delete a record of the database. Uses the primary key for identifying the record to be removed. Returns a table object.

Charlie decides to not travel in time. He exits our little group.

In [249]:
users.delete('Charlie')

<Table user (name, email, year_started)>

If the primary key value can't be found, raises a `NotFoundError`.

In John's case, he isn't time travelling with us yet so can't be removed.

In [250]:
try: users.delete('John')
except NotFoundError: print('User not found')

User not found


## .xtra()

The xtra action adds a filter to queries and DDL statements. This makes it easier to limit users (or other objects) access to only things for which they have permission. 

For example, if we query all our records below, you can see todos for everyone.

In [251]:
for o in todos():
    print(o)

Todo(id=1, title='Write MiniDataAPI spec', detail=None, status='open', name='Braden')
Todo(id=2, title='Implement SSE in FastHTML', detail=None, status='open', name='Alma')
Todo(id=3, title='Launch FastHTML', detail=None, status='closed', name='Charlie')


Now we use `.xtra` to constrain results just to Charlie.

In [252]:
todos.xtra(name='Alma')

Now if we loop over all the records again, only those assigned to Charlie will be displayed.

In [253]:
for o in todos():
    print(o)

Todo(id=2, title='Implement SSE in FastHTML', detail=None, status='open', name='Alma')


## Implementations

- [fastlite](https://github.com/AnswerDotAI/fastlite) - The original implementation, only for Sqlite
- [fastsql](https://github.com/AnswerDotAI/fastsql) - An SQL database agnostic implementation based on the excellent SQLAlchemy library.